In [22]:
import json
import cv2
from pathlib import Path
from albumentations import (
    Compose,
    RandomFog,
    RandomRain,
    RandomSunFlare,
    Blur,
    RandomSnow,
)
json_path = Path("test_labels.json")
input_dir = Path("test")           
min_width = 150
min_height = 150
augmentations = {
    "fog": Compose([
        RandomFog(
            alpha_coef=0.2,      
            p=1.0
        )
    ]),

    "rain": Compose([
        RandomRain(
            drop_color=[200, 200, 200],
            blur_value=15,          
            brightness_coefficient=1.0,  
            rain_type="heavy",      #
            p=1.0
        )
    ]),

    "sun_flare": Compose([
        RandomSunFlare(
            flare_roi=[0.0, 0.0, 1.0, 0.6], 
            src_radius=350,               
            src_color=[255, 255, 255],           
            glare_coefficient=0.5,           
            p=1.0
        )
    ]),

    "blur": Compose([
        Blur(
            blur_limit=(21, 21), 
            p=1.0
        )
    ]),

    "snow": Compose([
        RandomSnow(
            brightness_coeff=3.0,  
            p=1.0
        )
    ]),
    "grayscale": Compose([ToGray(p=1.0)]),
}
with open(json_path, "r") as f:
    label_dict = json.load(f)
data = []
for filename, label in label_dict.items():
    filename = filename.strip()
    label = label.strip()
    data.append({"filename": filename, "label": label})

print(f"Parsed {len(data)} entries from label file.")
print("First 3 entries:", data[:3])

for aug_name, aug_pipeline in augmentations.items():
    print(f"Applying {aug_name} augmentation…")
    output_dir = Path(aug_name)
    output_dir.mkdir(parents=True, exist_ok=True)

    for entry in data:
        img_path = input_dir / entry["filename"]
        img = cv2.imread(str(img_path))
        if img is None:
            print(f"Warning: could not read {img_path}")
            continue
        h, w = img.shape[:2]
        if h < min_height or w < min_width:
            # skip super small images
            continue
        augmented_img = aug_pipeline(image=img)["image"]
        save_path = output_dir / entry["filename"]
        cv2.imwrite(str(save_path), augmented_img)

print("All done—check the “augmented” subfolders for fog, rain, sun_flare, blur, and snow.")


/tmp/ipykernel_1891014/672654799.py:35: UserWarning: Argument(s) 'glare_coefficient' are not valid for transform RandomSunFlare
  RandomSunFlare(


Parsed 1335 entries from label file.
First 3 entries: [{'filename': '000000_0.png', 'label': 'Pedestrian'}, {'filename': '000006_0.png', 'label': 'Car'}, {'filename': '000008_2.png', 'label': 'Car'}]
Applying fog augmentation…
Applying rain augmentation…
Applying sun_flare augmentation…
Applying blur augmentation…
Applying snow augmentation…
Applying grayscale augmentation…
All done—check the “augmented” subfolders for fog, rain, sun_flare, blur, and snow.


In [17]:
import gradio as gr
import torch
import torch.nn as nn
from torchvision import models, transforms
from PIL import Image

def load_model():
    model = /path/to/your/model.pth  
    model.eval()
    return model
model = load_model()
ttt_model = load_model()  
transform = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.ToTensor(),
    transforms.Normalize(
        mean=[0.485, 0.456, 0.406], 
        std=[0.229, 0.224, 0.225]
    )
])

idx_to_label = {
    207: "Golden Retriever",
    281: "Tabby Cat",
    282: "Tiger Cat",
    409: "Orange",
    555: "Violin"
}

def infer(img: Image.Image):
    x = transform(img).unsqueeze(0)
    with torch.no_grad():
        out1 = model(x)
        out2 = ttt_model(x) 
    pred1 = torch.argmax(out1, dim=1).item()
    pred2 = torch.argmax(out2, dim=1).item()

    label1 = idx_to_label.get(pred1, f"Class {pred1}")
    label2 = idx_to_label.get(pred2, f"Class {pred2} (TTT)")

    return label1, label2

gr.Interface(
    fn=infer,
    inputs=gr.Image(type="pil"),
    outputs=["text", "text"],
    title="Test-Time Training Demo",
    description="Upload an image and see how TTT improves predictions (simulated)"
).launch()


/home/dice/anaconda3/envs/pytorch_env/lib/python3.9/site-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/home/dice/anaconda3/envs/pytorch_env/lib/python3.9/site-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=ResNet18_Weights.IMAGENET1K_V1`. You can also use `weights=ResNet18_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)


Running on local URL:  http://127.0.0.1:7860

To create a public link, set `share=True` in `launch()`.
